In [2]:
import pytest
import sys
import os 
import pandas as pd
import importlib
%matplotlib inline
sys.path.append('/workspace')
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  

In [3]:
from modules.Pipelines.FlowsPipe import FlowsPipe
from modules.Pipelines.NetworkDataPipe import NetworkDataPipe
from modules.Pipelines.ERCsAggregatedPipe import ERCsAggregatedPipe
from modules.Transformers.Aggregators.NetworkDataMetricsAggregator import NetworkDataMetricsAggregator
from modules.Transformers.Aggregators.NetworkDataXDayAverage import NetworkDataXDayAverage
from modules.Transformers.PercentGrowth import PercentGrowth
from modules.Visualizers.ERCs.LineChartERCsAggregateMetric import LineChartERCsAggregateMetric
from modules.Visualizers.Table import Table
from modules.Visualizers.LineChart import LineChart
from modules.Visualizers.LineChartMetricByAssets import LineChartMetricByAssets
from modules.Visualizers.MarketShareAssetsByMetric import MarketShareAssetsByMetric
from modules.Transformers.Rankers.RankMetricByAssets import RankMetricByAssets

In [25]:
api_key = 'KKzV6V2DTY87v3m1dGZu'
test_assets = ['btc', 'eth', 'mkr', 'bat', 'dcr', 'bch', 'ltc', 'bsv', 'ada', 'dash', 'xlm', 'trx', 'link', 'xtz']
stablecoins = ['dai', 'gusd', 'tusd', 'usdc', 'pax', 'usdt_eth']
start = '2019-01-01'
metrics = ['AdrBalCnt','AdrActCnt', 'AdrBal1in1BCnt','AdrBalUSD10Cnt','CapMrktCurUSD', 'CapRealUSD','IssTotNtv', 'SplyCur', 'TxCnt', 'TxTfrCnt', 'TxTfrValAdjUSD', 'VtyDayRet30d']


In [5]:
pipe = NetworkDataPipe(api_key=api_key, assets=test_assets, metrics=metrics, start=start)
df = pipe.run_pipeline().get_data()

In [16]:
table_df = NetworkDataXDayAverage(number_of_days=7).transform(df)
ranking_metrics = ['AdrActCnt','TxCnt', 'TxTfrCnt', 'TxTfrValAdjUSD', 'CapRealUSD', 'CapMrktCurUSD', 'SplyCur']
for ranking_metric in ranking_metrics:
    ranked = RankMetricByAssets(metric=ranking_metric).transform(table_df)
    top_10 = ranked.loc[0:9]
    top_10_assets = list(top_10['asset'])
    print(top_10_assets)
#     top_10_pipe = ERCsAggregatedPipe(api_key=api_key, ERCs=top_10_assets, metrics=metrics, start=start, aggregate_ERCs=False)
#     top_10_df = pipe.run_pipeline().get_data()
#     top_10_table = NetworkDataXDayAverage(number_of_days=30).transform(top_10_df)
#     market_share_chart = MarketShareAssetsByMetric(title='Share of ERC-20 Transactions, 2019'.format(ranking_metric), metric=ranking_metric, assets=top_10_assets).run(top_10_df)
#     market_share_chart.fig.show()


['btc', 'eth', 'trx', 'dash', 'xlm', 'ltc', 'bsv', 'bch', 'dcr', 'ada']
['trx', 'eth', 'bsv', 'xlm', 'btc', 'xtz', 'bch', 'ltc', 'dash', 'dcr']
['btc', 'bsv', 'eth', 'trx', 'dash', 'ltc', 'bch', 'xlm', 'dcr', 'link']
['btc', 'eth', 'bch', 'ada', 'ltc', 'bsv', 'xlm', 'mkr', 'link', 'dcr']
['btc', 'eth', 'xlm', 'bch', 'ltc', 'ada', 'dash', 'xtz', 'bsv', 'link']
['btc', 'eth', 'xlm', 'bch', 'ltc', 'link', 'bsv', 'ada', 'xtz', 'mkr']


ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

In [26]:
mkr_pipe = NetworkDataPipe(api_key=api_key, assets=['mkr'], metrics=metrics, start=start)
mkr_df = mkr_pipe.run_pipeline().get_data()

In [18]:
dai_pipe = NetworkDataPipe(api_key=api_key, assets=['dai'], metrics=metrics, start=start)
dai_df = dai_pipe.run_pipeline().get_data()

In [20]:
LineChartMetricByAssets(title='DAI Supply, 2019', metric='SplyCur', assets=['dai']).run(dai_df).fig.show()

In [14]:
for col in list(dai_df.columns):
    metric = col.split('.')[1]
    LineChartMetricByAssets(title=metric, metric=metric, assets=['dai']).run(dai_df).fig.show()

In [27]:
for col in list(mkr_df.columns):
    metric = col.split('.')[1]
    LineChartMetricByAssets(title=metric, metric=metric, assets=['mkr']).run(mkr_df).fig.show()